### 3.3. Representación semántica y aprendizaje de tópicos

#### 3.3.1. Embeddings multilingües

Los documentos $d_i$ se convierten en vectores densos mediante un bi-encoder multilingüe $\phi(\cdot)$ (Sentence-BERT). Para cada texto normalizado $x_i$, obtenemos:

$$
e_i = \phi(x_i) \in \mathbb{R}^p,\quad \|e_i\|_2 = 1
$$

La normalización L2 facilita que la similitud coseno sea equivalente al producto punto $e_i^\top e_j$. La elección de paraphrase-multilingual-mpnet-base-v2 responde a su buen desempeño en tareas de semantic textual similarity en múltiples idiomas, requisito para un corpus hispano–anglo heterogéneo.

#### 3.3.2. Reducción y clusterización densa

La matriz de embeddings $E \in \mathbb{R}^{n \times p}$ se reduce con UMAP a $Z \in \mathbb{R}^{n \times m}$ (con $m=10$) para preservar estructura de vecindad:

$$
Z = \text{UMAP}_m(E; n\_neighbors,\, min\_dist,\, metric)
$$

Luego, HDBSCAN detecta clústeres densos y permite un ruido $(-1)$ explícito. A diferencia de K-means, HDBSCAN no requiere fijar $k$ y es robusto a formas no convexas y densidades heterogéneas; su “prediction\_data” habilita asignación aproximada de nuevos puntos y re-etiquetación del ruido por fortaleza de pertenencia.

#### 3.3.3. BERTopic y representaciones de tópico

Con el mismo pipeline (embedding→UMAP→HDBSCAN), BERTopic construye tópicos $T = \{t\}$ y, para cada $t$, calcula palabras representativas (c-TF-IDF) y probabilidades documento–tópico $P(t|d)$. Para estabilizar centroides, el vector del tópico se define como:

$$
c_t = \frac{1}{|S_t|} \sum_{i \in S_t} e_i,\quad S_t = \text{TopK}(\{i: y_i = t\},\, k \text{ por } P(t|d_i))
$$

La visualización 2D usa una paleta HSL para $t \geq 0$ y gris para $-1$, mejorando la lectura perceptual de continuidad temática y separando el ruido.

### 3.4. Mapeo de tópicos a categorías mediante fusión semántica y NLI

Para vincular tópicos con categorías exógenas (p. ej., áreas disciplinares), se calculan prototipos de categoría a partir de centroides de embeddings de documentos etiquetados:

$$
k_c = \frac{1}{|D_c|} \sum_{i \in D_c} e_i
$$

La similitud bi-encoder tópico–categoría es $\cos(c_t, k_c)$. En paralelo, se formula una hipótesis textual para cada par $(t, c)$: “El tópico $t$ pertenece a la categoría $c$” (con descriptores TF-IDF) y se estima su probabilidad de implicación con un CrossEncoder NLI:

$$
s_{t,c}^{NLI} = P(\text{entailment} \mid \text{premisa}=t,\, \text{hipótesis}=c)
$$

La puntuación final que guía la asignación se fusiona por convexidad:

$$
S_{t,c} = \alpha \cos(c_t, k_c) + (1-\alpha) s_{t,c}^{NLI},\quad \alpha \in [0,1]
$$

Se obtienen salidas Top-1 y multietiqueta con umbral $\tau$. Los hiperparámetros $(\alpha, \tau, k)$ se afinan con Optuna (pruner SHA) maximizando una combinación de $F_1$ macro para Top-1 y hit-rate multietiqueta, siguiendo buenas prácticas de optimización bayesiana/búsqueda secuencial en ML Ops.
